In [ ]:
%%sh
pygmentize sklearn-boston-housing.py

In [ ]:
%%sh
pygmentize Dockerfile

In [ ]:
%%sh
aws ecr create-repository --repository-name sklearn-custom

In [ ]:
%%sh
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:sklearn -f Dockerfile .
export IMAGE_ID=`docker images -q sklearn-custom:sklearn`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn
aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:latest
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn

In [ ]:
import sagemaker
from sagemaker.sklearn import SKLearn

print(sagemaker.__version__)

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
sk = SKLearn(entry_point='sklearn-boston-housing.py',
             image_uri=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:sklearn',
             role=role,
             framework_version='0.20.0',
             instance_count=1, 
             instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training})

In [ ]:
%%sh
tar tvfz model.tar.gz